<a href="https://colab.research.google.com/github/convin305/sum-tudy/blob/master/%5Bdaycon%5Defficient_tta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install efficientnet_pytorch
!pip install git+https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp37-none-any.whl size=16031 sha256=756fc63109ae3c33d382c78df645a6456d2698226e55c9a2b8aa394a4a56592f
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch
  Cloning https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git to /tmp/pip-req-build-5i0ds6v6
  Running command git clone -q https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git /tmp/pip-req-build-5i0ds6v6
  Created wheel for torch-poly-lr-decay: filename=torch_poly_lr_decay-0.0.1-cp37-none-any.whl size=3833 sha256=f0cd91dd3c600af31e89770bc034b97018e50f61476e2dffb9bc5f9da826fdd4
  Stored in directory: /tmp/pip-ephem-wheel-cache-34eysjx0/wheels/5a/b7/09/d748b20c9bdfc768a33c37a28b2ad7dd9ec3e79e5152cb1618
Successfully built torch-poly-lr-decay


In [ ]:
import torch
import glob
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from tqdm import tqdm
import numpy as np
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import KFold
import time
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
from torch_poly_lr_decay import PolynomialLRDecay
import random

import torch.optim as optim


torch.set_num_threads(1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Dataset Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import output

!mkdir "./dirty_mnist"
!unzip "/content/drive/MyDrive/Data/dirty_mnist_2nd.zip" -d "./dirty_mnist/"

!mkdir "./test_dirty_mnist"
!unzip "/content/drive/MyDrive/Data/test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"

output.clear()

In [ ]:
labels_df = pd.read_csv('/content/drive/MyDrive/Data/dirty_mnist_2nd_answer.csv')[:]
imgs_dir = np.array(sorted(glob.glob('./dirty_mnist/*')))[:]
labels = np.array(labels_df.values[:,1:])
test_imgs_dir = np.array(sorted(glob.glob('./test_dirty_mnist/*')))

In [ ]:
test_imgs_dir

array(['./test_dirty_mnist/50000.png', './test_dirty_mnist/50001.png',
       './test_dirty_mnist/50002.png', ...,
       './test_dirty_mnist/54997.png', './test_dirty_mnist/54998.png',
       './test_dirty_mnist/54999.png'], dtype='<U28')

In [ ]:
'''imgs=[]
for path in imgs_dir[:]:
    img=cv2.imread(path, cv2.IMREAD_COLOR)
    imgs.append(img)'''

'imgs=[]\nfor path in imgs_dir[:]:\n    img=cv2.imread(path, cv2.IMREAD_COLOR)\n    imgs.append(img)'

In [ ]:
# 저장소에서 load
class MnistDataset_v1(Dataset):
    def __init__(self, imgs_dir=None, labels=None, transform=None, train=True):
        self.imgs_dir = imgs_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs_dir)
    
    def __getitem__(self, idx):
        # 1개 샘플 get
        img = cv2.imread(self.imgs_dir[idx], cv2.IMREAD_COLOR)
        img = self.transform(img)
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img
        
        pass
    


# 메모리에서 load
class MnistDataset_v2(Dataset):
    def __init__(self, imgs=None, labels=None, transform=None, train=True):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
        self.train=train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # 1개 샘플 get1
        img = self.imgs[idx]
        img = self.transform(img)
        
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img

#  SEED 설정

In [ ]:
# https://dacon.io/competitions/official/235697/codeshare/2363?page=1&dtype=recent&ptype=pub
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  

# Model 정의

In [ ]:
# EfficientNet -b0(pretrained)
# MultiLabel output

class EfficientNet_MultiLabel(nn.Module):
    def __init__(self, in_channels):
        super(EfficientNet_MultiLabel, self).__init__()
        self.network = EfficientNet.from_pretrained('efficientnet-b0', in_channels=in_channels)
        self.output_layer = nn.Linear(1000, 26)

    def forward(self, x):
        x = F.relu(self.network(x))
        x = torch.sigmoid(self.output_layer(x))
        return x
      

# 데이터 분리
* 1fold만 실행

In [ ]:
# 해당 코드에서는 1fold만 실행

kf = KFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in kf.split(imgs_dir):
    folds.append((train_idx, valid_idx))

In [ ]:
### seed_everything(42)

for fold in range(1):
    model = EfficientNet_MultiLabel(in_channels=3).to(device)
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]


    train_transform = transforms.Compose([
        transforms.ToTensor(),
        #transforms.RandomHorizontalFlip(),
        #transforms.RandomVerticalFlip(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225])
        ])
    valid_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225])
        ])

    epochs=20
    batch_size=32       # 자신의 VRAM에 맞게 조절해야 OOM을 피할 수 있습니다.
    
    
    
    # Data Loader
    train_dataset = MnistDataset_v1(imgs_dir = imgs_dir[train_idx], labels=labels[train_idx], transform=train_transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = MnistDataset_v1(imgs_dir = imgs_dir[valid_idx], labels = labels[valid_idx], transform=valid_transform)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)       
    
    
    # optimizer
    # polynomial optimizer를 사용합니다.
    # 
    optimizer = torch.optim.SGD(model.parameters(), lr=100)
    decay_steps = (len(train_dataset)//batch_size)*epochs
    scheduler_poly_lr_decay = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.1,step_size_up=5,mode="exp_range",gamma=0.85)

    criterion = torch.nn.BCELoss()
    
    
    epoch_accuracy = []
    valid_accuracy = []
    valid_losses=[]
    valid_best_accuracy=0
    for epoch in range(epochs):
        model.train()
        batch_accuracy_list = []
        batch_loss_list = []
        start=time.time()
        for n, (X, y) in enumerate((train_loader)):
            X = torch.tensor(X, device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            y_hat = model(X)
            
            
            optimizer.zero_grad()
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            scheduler_poly_lr_decay.step()

            
            y_hat  = y_hat.cpu().detach().numpy()
            y_hat = y_hat>0.5
            y = y.cpu().detach().numpy()

            batch_accuracy = (y_hat == y).mean()
            batch_accuracy_list.append(batch_accuracy)
            batch_loss_list.append(loss.item())

        model.eval()
        valid_batch_accuracy=[]
        valid_batch_loss = []
        with torch.no_grad():
            for n_valid, (X_valid, y_valid) in enumerate((valid_loader)):
                X_valid = torch.tensor(X_valid, device=device)#, dtype=torch.float32)
                y_valid = torch.tensor(y_valid, device=device, dtype=torch.float32)
                y_valid_hat = model(X_valid)
                
                valid_loss = criterion(y_valid_hat, y_valid).item()
                
                y_valid_hat = y_valid_hat.cpu().detach().numpy()>0.5
                
                
                valid_batch_loss.append(valid_loss)
                valid_batch_accuracy.append((y_valid_hat == y_valid.cpu().detach().numpy()).mean())
                
            valid_losses.append(np.mean(valid_batch_loss))
            valid_accuracy.append(np.mean(valid_batch_accuracy))
            
        if np.mean(valid_batch_accuracy)>valid_best_accuracy:
            torch.save(model.state_dict(), '/content/drive/MyDrive/Daycon/EfficientNetB0-fold{}.pt'.format(fold))
            valid_best_accuracy = np.mean(valid_batch_accuracy)
        print('fold : {}\tepoch : {:02d}\ttrain_accuracy / loss : {:.5f} / {:.5f}\tvalid_accuracy / loss : {:.5f} / {:.5f}\ttime : {:.0f}'.format(fold+1, epoch+1,
                                                                                                                                              np.mean(batch_accuracy_list),
                                                                                                                                              np.mean(batch_loss_list),
                                                                                                                                              np.mean(valid_batch_accuracy), 
                                                                                                                                              np.mean(valid_batch_loss),
                                                                                                                                              time.time()-start))

Loaded pretrained weights for efficientnet-b0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


KeyboardInterrupt: ignored

In [ ]:
test_transform = transforms.Compose([
        transforms.ToTensor(),
        ])

In [ ]:
torch.save(model.state_dict(),'~pt')

In [ ]:
#torch.load with map_location=torch.device('cpu')

submission = pd.read_csv('/content/drive/MyDrive/Data/sample_submission.csv')

with torch.no_grad():
      for fold in range(1): #5로 수정필요
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('/content/drive/MyDrive/Daycon/EfficientNetB0-fold{}.pt'.format(fold),map_location=torch.device('cpu')))
        model.eval()

        test_dataset = MnistDataset_v1(imgs_dir = test_imgs_dir, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission.iloc[n*32:(n+1)*32,1:] += pred_test

  0%|          | 0/157 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
100%|██████████| 157/157 [00:20<00:00,  7.77it/s]


In [ ]:
submission.iloc[:,1:] = np.where(submission.values[:,1:]>=0.5, 1,0)
submission.to_csv('EfficientNetB0_withELR-fold2,epoch10.csv', index=False) #저장확인하기!

In [ ]:
from google.colab import files

files.download("EfficientNetB0_withELR-fold2,epoch10.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>